<a href="https://colab.research.google.com/github/tgarnier067/MNLP-project-2/blob/main/02_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving the_vampyre_clean.json to the_vampyre_clean.json
Saving the_vampyre_ocr.json to the_vampyre_ocr.json


In [2]:
import json

with open("the_vampyre_clean.json", "r", encoding="utf-8") as f:
    clean_data = json.load(f)

with open("the_vampyre_ocr.json", "r", encoding="utf-8") as f:
    ocr_data = json.load(f)

# Prepare data

In [3]:
def concat_values_dict(d):
    """
    Concat values of a dict, seperating each element with '\n'

    Args:
        d (dict): Dictionnary

    Returns:
        str: concatenated text
    """
    return '\n'.join(d.get(str(i), "") for i in range(48))

clean_data_text = concat_values_dict(clean_data)

Possible models that we can try : Transducteurs (seq2seq with attention or transformers)

T5-small / T5-base

FLAN-T5-small

BART or distilBART

ByT5 : specialisez in byte-level treatment -> usefull for OCR errors.

Charformer / CANINE : char-level to get thine level of granularity

🔹 Baselines :
Levenshtein-based correction

Spell checkers + N-grams (as Hunspell or SymSpell with context)

## T5

In [4]:
!pip install transformers torch sentencepiece --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00


We create a function to apply a prompt wich asks to correct the data, to the LLM T5-base, and print the output of this LLM

In [96]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Charge model and tokenizer
model_name = "t5-base" # We could use also t5-small
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def correct_text_with_t5(text):
    prompt = f"Fix errors : {text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=False, padding=False).to(device)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [60]:
# Take the firsts sentences of a noisy sentence
noisy_text = ocr_data['0'][:1000]

# Build segment, and prompt to correct on each segment
segments = noisy_text.split('\n')

segments

['THE VAMPYRE;',
 'A Tale.',
 'By John William Polidori',
 'THEsuperstition upon which this taIe iſ founded is very general in the East. Among tho Arabjans itappeors to be common: it did not, however, extend itself to the Greeks until after the establi shment of Christianity; and it has only aſsumed its prosent form since the division af the Latin and Greok churches; at which time, lhe idea becoming prevalent, that a Lcltin body could not corrvpl if buried in their territory, it gradually increosed, and formed lhe subject of many wonderful stories, ſtill extant, of the dead rising from their graves, and feeding uponlhe blood of tho young and beautiful. In the West itspread, with some slight variation, all over Hungary, Poland, Austria, and Lorraine, whoro the helies existed, that vompyresnightly imbi6ed a certain portion of the blood of their victims, who became emaciated, lost their strength, and speedily died of c0nsumptions; whilst these human blood-suckers fattened—and their veins 

In [99]:
# Apply the model on a small dataset
for i in range(4):
  print(correct_text_with_t5(segments[i]))

.: THE VAMPYRE; THE VAMPYRE; THE VAMPY
: A Tale.
Fix errors : By John William Polidori : By John William Polidori :
. : Fix: Fix bug fixes : Fix errors : Fix bug fixes


This model makes a lot of problem, as you can see on the transcription he proposed on the above cell. I tried to modify the prompt, the lenght of the inputs, and many other parameters, but still, the quality of the output data was too bad. We will not focus on this model, but look at another one instead : vennify/t5-base-grammar-correction

In [62]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load fine-tuned grammar correction model
model_name = "vennify/t5-base-grammar-correction"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def correct_text_with_t5(text):
    prompt = f"correct: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test on a small dataset
noisy_text = ocr_data['0'][:1000]
segments = [s.strip() for s in noisy_text.split('\n') if s.strip()]

corrected_segments = [correct_text_with_t5(seg) for seg in segments]
corrected_text = '\n'.join(corrected_segments)

print("Corrected Output:\n", corrected_text)

Corrected Output:
 THE VAMPYRE.
A Tale.
By John William Polidori.
The superstition upon which this taIe is founded is very general in the East. It did not, however, extend itself to the Greeks until after the establishment of Christianity; and it has only assumed its prosent form since the division of the Latin and Greok churches; and it gradually increosed, and formed the subject of many wonderful stories, still extant, of the dead rising from their graves, and feeding upon the blood of young and beautiful people. In the West it spread, with some slight variation,


Problem : The output is not completed. So we increase the max_lenghts to 512. Unfortunatly, it's not enough => we have no longer output for very high values of max_lengths, than with max_lenght = 512. We also try to remove early_stopping, but it does not work. So, as the model can not output very long sentences, we apply it many times, on slices of the text.

- Instead of : model(sentence 1, sentence 2, sentence 3...)
- We do : model(sentence 1) + model(sentence 2) + model(sentence 3) + ...

We use spacy to slice into sentences

In [92]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import spacy

# Charger spaCy pour le découpage en phrases
nlp = spacy.load("en_core_web_sm")

# Charger le modèle T5 fine-tuné pour la correction grammaticale
model_name = "vennify/t5-base-grammar-correction"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fonction pour corriger une phrase avec T5
def correct_text_with_t5(text):
    prompt = f"correct: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_length=512,
        num_beams=4,
        early_stopping=False,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Découper un bloc de texte en phrases avec spaCy
def split_into_sentences_spacy(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

# Fonction principale : découpe par ligne, puis phrase, puis correction
def correct_text_by_line_and_sentence(text):
    lines = text.split('\n')
    corrected_lines = []

    for line in lines:
        if not line.strip():
            corrected_lines.append('')
            continue

        sentences = split_into_sentences_spacy(line)
        corrected_sentences = [correct_text_with_t5(sentence) for sentence in sentences]
        corrected_line = ' '.join(corrected_sentences)
        corrected_lines.append(corrected_line)

    return '\n'.join(corrected_lines)

# Exemple avec texte OCR
noisy_text = ocr_data['0'][:3000]  # ou le texte entier
corrected_text = correct_text_by_line_and_sentence(noisy_text)

print("Corrected Output:\n", corrected_text)


Corrected Output:
 THE VAMPYRE.
A Tale.
By John William Polidori.
THE superstition upon which this theory is founded is very general in the East. It did not, however, extend itself to the Greeks until after the establishment of Christianity; and it has only assumed its prosent form since the division of the Latin and Greok churches; at which time, the idea becoming prevalent, that a Lcltin body could not corrvpl if buried in their territory, gradually increased, and formed the subject of many wonderful stories, still extant, of the dead rising from their graves, and feeding upon the blood of young and beautiful. In the West it spread, with some slight variation, all over Hungary, Poland, Austria, and Lorraine, whoro the helies existed, that vompyresnightly imbi6ed a certain portion of the blood of their victims, who became emaciated, lost their strength, and quickly died of c0nsumptions; while these human blood-suckers fattened—and their veins became distended to such a state of roplet

In [93]:
len(corrected_text)

2733

In [95]:
print(clean_data_text[:3000])

THE VAMPYRE;
A Tale.
By John William Polidori
THE superstition upon which this tale is founded is very general in the East. Among the Arabians it appears to be common: it did not, however, extend itself to the Greeks until after the establishment of Christianity; and it has only assumed its present form since the division of the Latin and Greek churches; at which time, the idea becoming prevalent, that a Latin body could not corrupt if buried in their territory, it gradually increased, and formed the subject of many wonderful stories, still extant, of the dead rising from their graves, and feeding upon the blood of the young and beautiful. In the West it spread, with some slight variation, all over Hungary, Poland, Austria, and Lorraine, where the belief existed, that vampyres nightly imbibed a certain portion of the blood of their victims, who became emaciated, lost their strength, and speedily died of consumptions; whilst these human blood-suckers fattened—and their veins became dist

Problem : the model skip some parts of the text. We input 3000 characters, and it output only 2733. When looking into details at the translation, we see that it skips some sentences. Maybe, if we try to apply the LLM on smaller slices of the text, we won't skip parts. NOw, we slices when there is a '\n', and when there is a ,:;!?

In [90]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import spacy
import re

# Charger spaCy pour le découpage en phrases
nlp = spacy.load("en_core_web_sm")

# Charger le modèle T5 fine-tuné pour la correction grammaticale
model_name = "vennify/t5-base-grammar-correction"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fonction pour corriger une phrase avec T5
def correct_text_with_t5(text):
    prompt = f"correct: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_length=1024,       # Allonge la sortie maximale (attention à la RAM GPU)
        num_beams=4,
        early_stopping=False, # Ne pas stopper la génération prématurément
        length_penalty=1.0,   # Garde des sorties de taille naturelle
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Découper un bloc de texte en phrases avec spaCy
def split_into_sentences_spacy(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]



def split_sentences_and_punct(text):
    # Split d'abord en phrases spaCy
    spacy_sents = split_into_sentences_spacy(text)

    # Ensuite split par ponctuation dans chaque phrase
    punct_split_sents = []
    for sent in spacy_sents:
        # Split sur virgule, point d'exclamation, point d'interrogation, point-virgule, deux-points
        parts = re.split(r'[,:;!?]', sent)
        parts = [p.strip() for p in parts if p.strip()]
        punct_split_sents.extend(parts)
    return punct_split_sents


# Fonction principale : découpe par ligne, puis phrase, puis correction
def correct_text_by_line_and_sentence(text):
    lines = text.split('\n')
    corrected_lines = []

    for line in lines:
        if not line.strip():
            corrected_lines.append('')
            continue

        sentences = split_sentences_and_punct(line)
        corrected_sentences = [correct_text_with_t5(sentence) for sentence in sentences]
        corrected_line = ' '.join(corrected_sentences)
        corrected_lines.append(corrected_line)

    return '\n'.join(corrected_lines)

# Exemple avec texte OCR
noisy_text = ocr_data['0'][:3000]  # ou le texte entier
corrected_text = correct_text_by_line_and_sentence(noisy_text)

print("Corrected Output:\n", corrected_text)

Corrected Output:
 THE VAMPYRE is correct.
A Tale.
By John William Polidori.
THE superstition upon which this theory is founded is very general in the East. Among Arabjans itappeors to be common. It did not. However, this is correct: however, the facts are correct. It extends itself to the Greeks until after the establishment of Christianity. And it has only assumed its prosent form since the division of the Latin and Greok churches. At which time is correct: at which time. The idea is becoming prevalent. That a Lcltin body could not be corrvpl if buried in their territory. It gradually increased. And formed the subject of many wonderful stories. The fact is that there are still a lot of fossils that are still extant. The dead are rising from their graves. And feeding on the blood of tho young and beautiful. In the West it spreads throughout the world. With some slight variation. All over Hungary, Hungary is correct. Poland is correct. Austria is correct. And Lorraine is correct. Whoro

In [81]:
print(ocr_data['0'][:3000])

THE VAMPYRE;
A Tale.
By John William Polidori
THEsuperstition upon which this taIe iſ founded is very general in the East. Among tho Arabjans itappeors to be common: it did not, however, extend itself to the Greeks until after the establi shment of Christianity; and it has only aſsumed its prosent form since the division af the Latin and Greok churches; at which time, lhe idea becoming prevalent, that a Lcltin body could not corrvpl if buried in their territory, it gradually increosed, and formed lhe subject of many wonderful stories, ſtill extant, of the dead rising from their graves, and feeding uponlhe blood of tho young and beautiful. In the West itspread, with some slight variation, all over Hungary, Poland, Austria, and Lorraine, whoro the helies existed, that vompyresnightly imbi6ed a certain portion of the blood of their victims, who became emaciated, lost their strength, and speedily died of c0nsumptions; whilst these human blood-suckers fattened—and their veins became distend

In [91]:
len(corrected_text)

3686

New problems : the time computation starts beeing very high : 3 min to apply on 3000 characters. When we will generalize it to the 48 texts, it's going to take hours. Moreover, the outputs are not correct :    

- Noisy : In theLond0n Journal, of March, 1732, is a curiovs, and, of course, credible account of a particular case of vampyrifin, which is stated to hove accurred at Madreyga, in Hungary.

- Cleaned : In theLond0n Journal. The month of March is correct. 1732, correct: 1732. Is a curiovs. Correct: and so on. Of course, of course. Credible account of a particular case of vampyrifin. Which is stated to have been accurred at Madreyga. In Hungary.

In [7]:
# Evaluation

!pip install jiwer --quiet
from jiwer import wer, cer

total_cer = 0
total_wer = 0

for i in range(48):
    ref = clean_data_text[i]
    hyp = results[str(i)]
    total_cer += cer(ref, hyp)
    total_wer += wer(ref, hyp)

print(f"Mean CER: {total_cer / 48:.4f}")
print(f"Mean WER: {total_wer / 48:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.8 MB/s eta 0:00:00


ValueError: After applying the transformation, each reference should be a non-empty list of strings, with each string being a single word.

# Ideas

Possible ideas : get some data from the internet, pass it through an OCR, get the noisy text, and use it to finetune a model ?